# **LifeHack 2020- Problem #2**

Vision Impairment can be a serious implication on one’s health. According to the key facts stated by the World Health Organization “Globally, at least 2.2 billion people have a vision impairment or blindness, of whom at least 1 billion have a vision impairment that could have been prevented or has yet to be addressed.” Do you have the ability to better detect potential vision issues and help change thousands, if not, millions of lives? 

For example, the given dataset looks at cataract and normal eye image dataset for cataract detection. Participants are encouraged to use other available datasets to better analyse and share their findings regarding the detection of the potential risk of visual problems detection, preferably relating to the Asia region. Indeed this problem statement covers a wide area of visual impairments and does not need to focus solely on cataract. Participants are free to explore other options of eye disease detection.  

* Present Insight into the data by mean of visualization and/or interactive means
* Prediction model for eye disease 

For reference:
https://www.kaggle.com/jr2ngb/cataractdataset?Vision 

In [8]:
import os
import numpy as np
import shutil
import matplotlib                  # 2D Plotting Library
import matplotlib.pyplot as plt
import seaborn as sns              # Python Data Visualization Library based on matplotlib
from sklearn.metrics import classification_report
#import geopandas as gpd            # Python Geospatial Data Library
plt.style.use('fivethirtyeight')
%matplotlib inline

In [9]:
# Importing all necessary libraries 
from keras.preprocessing.image import ImageDataGenerator 
from keras.preprocessing import image
from keras.models import Model, Sequential 
from keras.layers import Dense, Conv2D, GlobalAveragePooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import backend as K
from keras.optimizers import SGD

from keras.applications import ResNet50

In [10]:
img_width, img_height = 224, 224

In [11]:
#Creating Train / Val / Test folders 
root_dir = 'cataract_dataset_bundle_archive'
normal = '/1_normal'
cataract = '/2_cataract'
glaucoma = '/2_glaucoma'
retina_disease = '/3_retina_disease'

In [12]:
#Creating Train / Val / Test folders 
root_dir = 'cataract_dataset_bundle_archive'
classes = ['/1_normal', '/2_cataract', '/2_glaucoma', '/3_retina_disease']

for class_type in classes:
    os.makedirs(root_dir + '/train' + class_type)
    os.makedirs(root_dir + '/val' + class_type)
    os.makedirs(root_dir + '/test' + class_type)

    src = root_dir + class_type

    allFiles = os.listdir(src)
    np.random.shuffle(allFiles)
    train_Files, val_Files, test_Files = np.split(np.array(allFiles),
                                                          [int(len(allFiles)*0.7), int(len(allFiles)*0.85)])
                                                          
    train_Files = [src+'/'+ name for name in train_Files.tolist()]
    val_Files = [src+'/' + name for name in val_Files.tolist()]
    test_Files = [src+'/' + name for name in test_Files.tolist()]

    print('Class: ', class_type)
    print('Total images: ', len(allFiles))
    print('Training: ', len(train_Files))
    print('Validation: ', len(val_Files))
    print('Testing: ', len(test_Files))

    for name in train_Files:
        shutil.copy(name, root_dir + "/train" + class_type)
    
    for name in val_Files:
        shutil.copy(name, root_dir + "/val" + class_type)
    
    for name in test_Files:
        shutil.copy(name, root_dir + "/test" + class_type)

FileExistsError: [Errno 17] File exists: 'cataract_dataset_bundle_archive/train/1_normal'

### Pre-processed data is stored in a new directory - "Data"

**Running Resnet** 

In [13]:
train_data_dir = f'data/train'
validation_data_dir = f'data/val'
nb_train_samples = 420 
nb_validation_samples = 90
epochs = 10
batch_size = 32

In [14]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='categorical')

Found 420 images belonging to 4 classes.
Found 90 images belonging to 4 classes.


In [15]:
base_model = ResNet50(weights='imagenet', include_top=False)

In [16]:
base_model.summary()

n[0][0]          
__________________________________________________________________________________________________
conv3_block4_out (Activation)   (None, None, None, 5 0           conv3_block4_add[0][0]           
__________________________________________________________________________________________________
conv4_block1_1_conv (Conv2D)    (None, None, None, 2 131328      conv3_block4_out[0][0]           
__________________________________________________________________________________________________
conv4_block1_1_bn (BatchNormali (None, None, None, 2 1024        conv4_block1_1_conv[0][0]        
__________________________________________________________________________________________________
conv4_block1_1_relu (Activation (None, None, None, 2 0           conv4_block1_1_bn[0][0]          
__________________________________________________________________________________________________
conv4_block1_2_conv (Conv2D)    (None, None, None, 2 590080      conv4_block1_1_relu[0][0] 

In [17]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

In [18]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer=SGD(lr=1e-1, momentum=0.9, decay=1e-1 / epochs), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
# %%time
# model.fit_generator(train_generator, train_generator.n // batch_size, epochs=2, workers=4,
#         validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

ADAM seems to give the best results

In [20]:
test_data_dir = f'data/test'

testing_generator = test_datagen.flow_from_directory(test_data_dir,
    shuffle=False,
    target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='categorical')

Found 91 images belonging to 4 classes.


In [21]:
# test model
print("-------------------------------  TESTING MULTI CLASSIFICATION MODEL  ----------------------------------------")
testing_generator.reset()
prediction_index = model.predict_generator(testing_generator, steps=(91 // batch_size) + 1)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
prediction_index = np.argmax(prediction_index, axis=1)

# show a nicely formatted classification report
print(classification_report(testing_generator.classes, prediction_index, target_names=testing_generator.class_indices.keys()))

-------------------------------  TESTING MULTI CLASSIFICATION MODEL  ----------------------------------------
Instructions for updating:
Please use Model.predict, which supports generators.
                  precision    recall  f1-score   support

        1_normal       0.52      0.96      0.67        45
      2_cataract       0.00      0.00      0.00        15
      2_glaucoma       0.00      0.00      0.00        16
3_retina_disease       0.12      0.07      0.09        15

        accuracy                           0.48        91
       macro avg       0.16      0.26      0.19        91
    weighted avg       0.28      0.48      0.35        91



In [43]:
    train_data_dir = f'data/binary_classifiers/cataract/train'
    validation_data_dir = f'data/binary_classifiers/cataract/val'
    nb_train_samples = 280
    nb_validation_samples = 60
    epochs = 10
    batch_size = 8
    train_datagen = ImageDataGenerator(rescale=1. / 255,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size, class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(validation_data_dir,
        shuffle=False,
        target_size=(img_width, img_height),
        batch_size=batch_size, class_mode='categorical')

    base_model = ResNet50(weights='imagenet', include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(64, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)   

    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers: layer.trainable = False
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



Found 280 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [44]:
%%time
    model.fit_generator(train_generator, train_generator.n // batch_size, epochs=5, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/5
35/35 [==============================] - 54s 2s/step - loss: 0.6909 - accuracy: 0.7143 - val_loss: 0.4986 - val_accuracy: 0.8036
Epoch 2/5
35/35 [==============================] - 56s 2s/step - loss: 0.5786 - accuracy: 0.7500 - val_loss: 0.4991 - val_accuracy: 0.8036
Epoch 3/5
35/35 [==============================] - 58s 2s/step - loss: 0.5704 - accuracy: 0.7500 - val_loss: 0.4973 - val_accuracy: 0.8036
Epoch 4/5
35/35 [==============================] - 57s 2s/step - loss: 0.5886 - accuracy: 0.7500 - val_loss: 0.4985 - val_accuracy: 0.8036
Epoch 5/5
35/35 [==============================] - 56s 2s/step - loss: 0.5858 - accuracy: 0.7500 - val_loss: 0.4978 - val_accuracy: 0.8036
CPU times: user 13min 40s, sys: 1min 19s, total: 15min
Wall time: 5min 1s


In [51]:
test_data_dir = f'data/binary_classifiers/cataract/test'

testing_generator = test_datagen.flow_from_directory(test_data_dir,
    shuffle=False,
    target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='categorical')

# test model
print("-------------------------------  TESTING BINARY CLASSIFICATION MODEL FOR CATARACT  ----------------------------------------")
testing_generator.reset()
prediction_index = model.predict_generator(testing_generator, steps=(60 // batch_size) + 1)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
prediction_index = np.argmax(prediction_index, axis=1)

# show a nicely formatted classification report
print(classification_report(testing_generator.classes, prediction_index, target_names=testing_generator.class_indices.keys()))

Found 60 images belonging to 2 classes.
-------------------------------  TESTING BINARY CLASSIFICATION MODEL FOR CATARACT  ----------------------------------------
              precision    recall  f1-score   support

    1_normal       0.75      1.00      0.86        45
  2_cataract       0.00      0.00      0.00        15

    accuracy                           0.75        60
   macro avg       0.38      0.50      0.43        60
weighted avg       0.56      0.75      0.64        60



In [58]:
train_data_dir = f'data/binary_classifiers/glaucoma/train'
validation_data_dir = f'data/binary_classifiers/glaucoma/val'
nb_train_samples = 280
nb_validation_samples = 60
epochs = 10
batch_size = 8
train_datagen = ImageDataGenerator(rescale=1. / 255,
shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='categorical')

base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)   

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Found 280 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [53]:
%%time
    model.fit_generator(train_generator, train_generator.n // batch_size, epochs=5, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/5
35/35 [==============================] - 67s 2s/step - loss: 0.6128 - accuracy: 0.7143 - val_loss: 0.5058 - val_accuracy: 0.8036
Epoch 2/5
35/35 [==============================] - 63s 2s/step - loss: 0.6086 - accuracy: 0.7036 - val_loss: 0.5015 - val_accuracy: 0.8036
Epoch 3/5
35/35 [==============================] - 59s 2s/step - loss: 0.6038 - accuracy: 0.7500 - val_loss: 0.5591 - val_accuracy: 0.8036
Epoch 4/5
35/35 [==============================] - 59s 2s/step - loss: 0.5818 - accuracy: 0.7321 - val_loss: 0.5028 - val_accuracy: 0.8036
Epoch 5/5
35/35 [==============================] - 53s 2s/step - loss: 0.5787 - accuracy: 0.7500 - val_loss: 0.5313 - val_accuracy: 0.8036
CPU times: user 13min 36s, sys: 1min 19s, total: 14min 56s
Wall time: 5min 20s


In [ ]:
test_data_dir = f'data/binary_classifiers/glaucoma/test'

testing_generator = test_datagen.flow_from_directory(test_data_dir,
    shuffle=False,
    target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='categorical')

# test model
print("-------------------------------  TESTING BINARY CLASSIFICATION MODEL FOR GLAUCOMA  ----------------------------------------")
testing_generator.reset()
prediction_index = model.predict_generator(testing_generator, steps=(60 // batch_size) + 1)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
prediction_index = np.argmax(prediction_index, axis=1)

# show a nicely formatted classification report
print(classification_report(testing_generator.classes, prediction_index, target_names=testing_generator.class_indices.keys()))

In [62]:
train_data_dir = f'data/binary_classifiers/retina_disease/train'
validation_data_dir = f'data/binary_classifiers/retina_disease/val'
nb_train_samples = 280
nb_validation_samples = 60
epochs = 10
batch_size = 8
train_datagen = ImageDataGenerator(rescale=1. / 255,
shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='categorical')

base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)   

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Found 280 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [64]:
%%time
    model.fit_generator(train_generator, train_generator.n // batch_size, epochs=5, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/5
35/35 [==============================] - 59s 2s/step - loss: 0.6300 - accuracy: 0.7107 - val_loss: 0.5001 - val_accuracy: 0.8036
Epoch 2/5
35/35 [==============================] - 59s 2s/step - loss: 0.6045 - accuracy: 0.7143 - val_loss: 0.4999 - val_accuracy: 0.8036
Epoch 3/5
35/35 [==============================] - 60s 2s/step - loss: 0.5781 - accuracy: 0.7500 - val_loss: 0.5119 - val_accuracy: 0.8036
Epoch 4/5
35/35 [==============================] - 59s 2s/step - loss: 0.6202 - accuracy: 0.7500 - val_loss: 0.5488 - val_accuracy: 0.8036
Epoch 5/5
35/35 [==============================] - 56s 2s/step - loss: 0.5831 - accuracy: 0.7500 - val_loss: 0.5059 - val_accuracy: 0.8036
CPU times: user 13min 45s, sys: 1min 17s, total: 15min 2s
Wall time: 5min 10s


In [67]:
test_data_dir = f'data/binary_classifiers/retina_disease/test'

testing_generator = test_datagen.flow_from_directory(test_data_dir,
    shuffle=False,
    target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='categorical')

# test model
print("-------------------------------  TESTING BINARY CLASSIFICATION MODEL FOR RETINA DISEASES  ----------------------------------------")
testing_generator.reset()
prediction_index = model.predict_generator(testing_generator, steps=(60 // batch_size) + 1)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
prediction_index = np.argmax(prediction_index, axis=1)

# show a nicely formatted classification report
print(classification_report(testing_generator.classes, prediction_index, target_names=testing_generator.class_indices.keys()))

Found 60 images belonging to 2 classes.
-------------------------------  TESTING BINARY CLASSIFICATION MODEL FOR RETINA DISEASES  ----------------------------------------
                  precision    recall  f1-score   support

        1_normal       0.75      1.00      0.86        45
3_retina_disease       0.00      0.00      0.00        15

        accuracy                           0.75        60
       macro avg       0.38      0.50      0.43        60
    weighted avg       0.56      0.75      0.64        60

